In [1]:
import numpy as np
import phate
from heatgeo.embedding import HeatGeo
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm

dnames = []
for noise in [0.1,1.0]:
    for seed in [1,2]:
        dnames.append(f'swiss_roll_noise{noise}_seed{seed}')
        dnames.append(f's_curve_noise{noise}_seed{seed}')
        dnames.append(f'tree_noise{noise}_seed{seed}')
print(dnames)

for dname in tqdm(dnames):
# for dname in ['tree_noise1.0_seed2']:
    data = np.load(f"/gpfs/gibbs/pi/krishnaswamy_smita/dl2282/dmae/data/{dname}.npz")
    x = data['data']
    color = data['colors']
    phate_op = phate.PHATE(random_state=1, knn=5, n_landmark=5000, t=60)
    x_phate = phate_op.fit_transform(x)

    data_dict = {file: data[file] for file in data.files}
    data_dict['is_train'] = data_dict['is_train'].astype(bool)
    is_train = data_dict['is_train']
    data_dict['phate'] = x_phate
    phate_dist = squareform(pdist(phate_op.diff_potential))

    phate_op = phate.PHATE(random_state=1, knn=5, n_landmark=5000, t=60)
    _ = phate_op.fit_transform(x[is_train])
    dists_train = squareform(pdist(phate_op.diff_potential))
    phate_dist[is_train][:,is_train] = dists_train

    data_dict['dist'] = phate_dist
    np.savez(f"../data/{dname}.npz", **data_dict)


['swiss_roll_noise0.1_seed1', 's_curve_noise0.1_seed1', 'tree_noise0.1_seed1', 'swiss_roll_noise0.1_seed2', 's_curve_noise0.1_seed2', 'tree_noise0.1_seed2', 'swiss_roll_noise1.0_seed1', 's_curve_noise1.0_seed1', 'tree_noise1.0_seed1', 'swiss_roll_noise1.0_seed2', 's_curve_noise1.0_seed2', 'tree_noise1.0_seed2']
Calculating PHATE...
  Running PHATE on 3000 observations and 3 variables.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN search in 0.03 seconds.
    Calculating affinities...
    Calculated affinities in 0.02 seconds.
  Calculated graph and diffusion operator in 0.06 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 6.06 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 11.82 seconds.
Calculated PHATE in 17.99 seconds.
Calculating PHATE...
  Running PHATE on 2400 observations and 3 variables.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN

In [2]:
(phate_dist[is_train][:,is_train]).shape

(2000, 2000)

In [3]:
len(dnames)

12

In [4]:
phate_op.diff_potential.shape

(2000, 2000)